# Gravitational attraction between two bodies 
Defines an Orbit class that is used to generate 2-body orbit plots from solving Lagrange's equations in cartesian coordinates.

* Last revised 25-Apr-2019 by Abasi Brown (brown.7146@buckeyemail.osu.edu).

## Euler-Lagrange equation

For a 2-body orbit, the Lagrangian with generalized coordinates $x_{1}, y_{2}, x_{2},$ & $y_{2}$  is

$\begin{align}
  \mathcal{L} = \frac12 m_{1}\dot x_{1}^2 + \frac12 m_{1}\dot y_{1}^2 + \frac12 m_{2}\dot x_{2}^2 + \frac12 m_{2}\dot y_{2}^2 + \frac{Gm_{1}m_{2}}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac12}
\end{align}$

The Euler-Lagrange equations are:

$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot x_{1}} =  m_{1}\ddot x_{1}
  \;
\end{align}$

$\begin{align}
 \frac{\partial\mathcal L}{\partial x_{1}}
 = \frac{Gm_{1}m_{2}(x_{2} - x_{1})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$
 
$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot y_{1}} =  m_{1}\ddot y_{1}
  \;
\end{align}$

$\begin{align}
 \frac{\partial\mathcal L}{\partial y_{1}}
 = \frac{Gm_{1}m_{2}(y_{2} - y_{1})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$ 
 
 $\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot x_{2}} =  m_{2}\ddot x_{2}
  \;
\end{align}$

$\begin{align}
 \frac{\partial\mathcal L}{\partial x_{2}}
 = \frac{Gm_{1}m_{2}(x_{1} - x_{2})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$
 
$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot y_{2}} =  m_{2}\ddot y_{2}
  \;
\end{align}$

$\begin{align}
 \frac{\partial\mathcal L}{\partial y_{2}}
 = \frac{Gm_{1}m_{2}(y_{1} - y_{2})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$
 
 From these equations we obtain the following values for $\ddot x_{1}, \ddot y_{1}, \ddot x_{2},$ & $\ddot y_{2}$:

$\begin{align}
 \ddot x_{1}
 = \frac{Gm_{2}(x_{2} - x_{1})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$
 
 $\begin{align}
 \ddot y_{1}
 = \frac{Gm_{2}(y_{2} - y_{1})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$
 
 $\begin{align}
 \ddot x_{2}
 = \frac{Gm_{1}(x_{1} - x_{2})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$
 
 $\begin{align}
 \ddot y_{2}
 = \frac{Gm_{1}(y_{1} - y_{2})}{((x_{2}-x_{1})^2 + (y_{2}-y_{1})^2)^\frac32}
 \end{align}$

In [58]:
%matplotlib inline

In [59]:
# Math functions
import numpy as np

# Solve ODE
from scipy.integrate import odeint, solve_ivp

# Plotting
import matplotlib.pyplot as plt

# Widgets and display
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 
from IPython.display import display
from time import sleep
from matplotlib import animation, rc
from IPython.display import HTML

In [60]:
# The dpi (dots-per-inch) setting will affect the resolution and how large
#  the plots appear on screen and printed.  So you may want/need to adjust 
#  the figsize when creating the figure.
plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

## Orbit class

In [61]:
class Orbit():
    """
    Orbit class implements the parameters and Lagrange's equations for 
     a 2-body orbit.
     
    Parameters
    ----------
    m1 : float
        mass of object 1
    m2 : float
        mass of object 1
    G : float
        gravitational constant

    Methods
    -------
    du_dt(t, u)
        Returns the right side of the differential equation in vector u, 
        given time t and the corresponding value of u.
    """
    def __init__(self, m1=1., m2=1., G=1.):
        self.G = G
        self.m1 = m1
        self.m2 = m2
        
    def du_dt(self, t, u):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ---------- 
        u : float
            An 8-component vector with u[0] = x1, u[1] = x1_dot,
            u[2] = y1, u[3] = y1_dot, u[4] = x2, u[5] = x2_dot,
            u[6] = y2, and u[7] = y2_dot,
            
        Returns
        -------
        
        """
        r = np.sqrt((u[4] - u[0])**2 + (u[6] - u[2])**2)
        
        x1_ddot = self.G * self.m2 * (u[4] - u[0]) / r**3
        y1_ddot = self.G * self.m2 * (u[6] - u[2]) / r**3
        
        x2_ddot = self.G * self.m1 * (u[0] - u[4]) / r**3
        y2_ddot = self.G * self.m1 * (u[2] - u[6]) / r**3
        
        return [u[1], x1_ddot, u[3], y1_ddot, u[5], x2_ddot, u[7], y2_ddot]
    
    def solve_ode(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                  x2_0, x2_dot_0, y2_0, y2_dot_0, abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        u = [x1_0, x1_dot_0, y1_0, y1_dot_0, x2_0, x2_dot_0, y2_0, y2_dot_0] 
        
        solution = solve_ivp(self.du_dt, (t_pts[0], t_pts[-1]), 
                             u, t_eval=t_pts, method='RK23',
                             atol=abserr, rtol=relerr)
        
        x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = solution.y
        
        # Calculate energy 
        potential = -self.G * self.m1 * self.m2 / np.sqrt((x2 - x1)**2 + (y2 - y1)**2 )
        
        energy = (1/2) * self.m1 * x1_dot**2 + (1/2) * self.m1 * y1_dot**2 +\
                   (1/2) * self.m2 * x2_dot**2 + (1/2) * self.m2 * y2_dot**2 + potential
        
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, energy
    
    def solve_ode_Leapfrog(self, t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                          x2_0, x2_dot_0, y2_0, y2_dot_0):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for x1, x1_dot, y1, and y1_dot with zeros
        num_t_pts = len(t_pts)
        
        x1 = np.zeros(num_t_pts)
        x1_dot = np.zeros(num_t_pts)
        x1_dot_half = np.zeros(num_t_pts)
        
        y1 = np.zeros(num_t_pts)
        y1_dot = np.zeros(num_t_pts)
        y1_dot_half = np.zeros(num_t_pts)
        
        x2 = np.zeros(num_t_pts)
        x2_dot = np.zeros(num_t_pts)
        x2_dot_half = np.zeros(num_t_pts)
        
        y2 = np.zeros(num_t_pts)
        y2_dot = np.zeros(num_t_pts)
        y2_dot_half = np.zeros(num_t_pts)
        
        # initial conditions
        x1[0] = x1_0
        x1_dot[0] = x1_dot_0
        
        y1[0] = y1_0
        y1_dot[0] = y1_dot_0
        
        x2[0] = x2_0
        x2_dot[0] = x2_dot_0
        
        y2[0] = y2_0
        y2_dot[0] = y2_dot_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            
            u = [x1[i], x1_dot[i], y1[i], y1_dot[i], x2[i], x2_dot[i], y2[i], y2_dot[i]]
            
            x1_dot_half[i] = x1_dot[i] + self.du_dt(t, u)[1] * delta_t/2.
            x1[i+1] = x1[i] + x1_dot_half[i] * delta_t
            
            y1_dot_half[i] = y1_dot[i] + self.du_dt(t, u)[3] * delta_t/2.
            y1[i+1] = y1[i] + y1_dot_half[i] * delta_t
            
            x2_dot_half[i] = x2_dot[i] + self.du_dt(t, u)[5] * delta_t/2.
            x2[i+1] = x2[i] + x2_dot_half[i] * delta_t
            
            y2_dot_half[i] = y2_dot[i] + self.du_dt(t, u)[7] * delta_t/2.
            y2[i+1] = y2[i] + y2_dot_half[i] * delta_t
            
            u = [x1[i+1], x1_dot[i], y1[i+1], y1_dot[i], x2[i+1], x2_dot[i], y2[i+1], y2_dot[i]]
            
            x1_dot[i+1] = x1_dot_half[i] + self.du_dt(t, u)[1] * delta_t/2.
            y1_dot[i+1] = y1_dot_half[i] + self.du_dt(t, u)[3] * delta_t/2.
            x2_dot[i+1] = x2_dot_half[i] + self.du_dt(t, u)[5] * delta_t/2.
            y2_dot[i+1] = y2_dot_half[i] + self.du_dt(t, u)[7] * delta_t/2.
        
        # Calculate energy
        potential = -self.G * self.m1 * self.m2 / np.sqrt((x2 - x1)**2 + (y2 - y1)**2 )
        
        energy = (1/2) * self.m1 * x1_dot**2 + (1/2) * self.m1 * y1_dot**2 +\
                   (1/2) * self.m2 * x2_dot**2 + (1/2) * self.m2 * y2_dot**2 + potential
            
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, energy 

## Plotting functions

In [62]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

# Labels for individual plot axes
x_vs_y_labels = (r'$x(t)$', r'$y(t)$')
x_dot_vs_y_dot_labels = (r'$\dot x(t)$', r'$\dot y(t)$')
energy_labels = (r'$t$', r'$U(t)$')

## Set parameters and initial conditions

In [63]:
"""
These values are passed to the widgets so they only need
to be updated in this section
"""
# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 2.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Orbit parameters
m1 = 100.
m2 = 1.
G = 1.

# initial conditions
x1_0 = .5
x1_dot_0 = 0.
y1_0 = 0.
y1_dot_0 = .1

x2_0 = -m1 * x1_0 / m2
x2_dot_0 = -m1 * x1_dot_0 / m2
y2_0 = -m1 * y1_0 / m2
y2_dot_0 = -m1 * y1_dot_0 / m2

## Generate plots

In [64]:
#This function generates the main output, which is a grid of plots
def orbit_plots(x_vs_y_plot=True, x_dot_vs_y_dot_plot=True, 
                energy_plot=True, x_vs_y_plot_LF=True, x_dot_vs_y_dot_plot_LF=True, 
                energy_plot_LF=True, m1=m1, m2=m2, G=G, x1_0=x1_0, x1_dot_0=x1_dot_0, y1_0=y1_0,
                y1_dot_0=y1_dot_0, x2_0=x2_0, x2_dot_0=x2_dot_0, y2_0=y2_0, y2_dot_0=y2_dot_0,
                t_start=t_start, t_end=t_end, delta_t=delta_t, plot_start=0, x_vs_y_labels=x_vs_y_labels,
                x_dot_vs_y_dot_labels=x_dot_vs_y_dot_labels, energy_labels=energy_labels,
                animate_flag=False, t_index=0,font_size=18):
    """
    Create plots for interactive_output according to the inputs.
    
    Based on generating an Orbit instance and the requested graphs.
    """
    
    # add delta_t o it goes at least to t_end (probably should use linspace)
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)  
        
    # Instantiate a pendulum with the passed (or default) parameters
    O1 = Orbit(m1=m1, m2=m2, G=G)
    
    # Solve SciPy ODE
    x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot, energy = \
    O1.solve_ode(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                 x2_0, x2_dot_0, y2_0, y2_dot_0)

    # Solve Leapfrog ODE
    x1_LF, x1_dot_LF, y1_LF, y1_dot_LF, x2_LF, x2_dot_LF, y2_LF, y2_dot_LF, \
    energy_LF = \
    O1.solve_ode_Leapfrog(t_pts, x1_0, x1_dot_0, y1_0, y1_dot_0,
                x2_0, x2_dot_0, y2_0, y2_dot_0)
    
    # Figure out how many rows and columns
    plot_flags = [x_vs_y_plot, x_dot_vs_y_dot_plot, energy_plot,
                  x_vs_y_plot_LF, x_dot_vs_y_dot_plot_LF, energy_plot_LF]
    plot_num = plot_flags.count(True)
    if plot_num <= 3:
        plot_rows = 1
    else:
        plot_rows = 2
    figsize_rows = plot_rows*6
    if plot_num <= 3:
        plot_cols = plot_num
    elif plot_num == 4:
        plot_cols = 2
    else:
        plot_cols = 3
    figsize_cols = min(plot_cols*8, 16)  # at most 16
    
    # Make the plot!
    fig, axes = plt.subplots(plot_rows, plot_cols, 
                             figsize=(figsize_cols,figsize_rows))
    #axes = np.atleast_1d(axes)  # make it always a 1d array, even if only 1
    axes = np.ravel(axes)
    
    start_index = (np.fabs(t_pts-plot_start)).argmin() # finds nearest index
    
    next_axes = 0
    if x_vs_y_plot:
        plot_y_vs_x(x1, y1,
                    axis_labels=x_vs_y_labels, 
                    label=rf'$orbit_{1}$',
                    color = 'blue', 
                    title=r'Position', 
                    ax=axes[next_axes]) 
        plot_y_vs_x(x2, y2,
                    axis_labels=x_vs_y_labels, 
                    label=rf'$orbit_{2}$', 
                    color = 'red', 
                    title=r'Position', 
                    ax=axes[next_axes])
        if animate_flag:
            axes[next_axes].plot(x1[t_index], y1[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(x2[t_index], y2[t_index], 'ro', markersize = '16')
        axes[next_axes].set_aspect(1)
        next_axes += 1
    
    if x_dot_vs_y_dot_plot:
        plot_y_vs_x(x1_dot, y1_dot, 
                    axis_labels=x_dot_vs_y_dot_labels, 
                    label=rf'$orbit_{1}$', 
                    color = 'blue', 
                    title=r'Velocity', 
                    ax=axes[next_axes]) 
        plot_y_vs_x(x2_dot, y2_dot, 
                    axis_labels=x_dot_vs_y_dot_labels, 
                    label=rf'$orbit_{2}$', 
                    color = 'red', 
                    title=r'Velocity', 
                    ax=axes[next_axes])
        if animate_flag:
            axes[next_axes].plot(x1_dot[t_index], y1_dot[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(x2_dot[t_index], y2_dot[t_index], 'ro', markersize = '16')
        axes[next_axes].set_aspect(1)
        next_axes += 1

    if energy_plot:
        plot_y_vs_x(t_pts, energy, 
                    axis_labels=energy_labels,
                    title='Energy', 
                    color = 'green', 
                    ax=axes[next_axes])
        if animate_flag:
            axes[next_axes].plot(t_pts[t_index], energy[t_index], 'go', markersize = '16')
        next_axes += 1
        
    if x_vs_y_plot_LF:
        plot_y_vs_x(x1_LF, y1_LF,
                    axis_labels=x_vs_y_labels, 
                    label=rf'$orbit_{1}$',
                    color = 'blue',
                    title=r'Position Leapfrog Method', 
                    ax=axes[next_axes]) 
        plot_y_vs_x(x2_LF, y2_LF,
                    axis_labels=x_vs_y_labels, 
                    label=rf'$orbit_{2}$',
                    color = 'red',
                    title=r'Position Leapfrog Method', 
                    ax=axes[next_axes])
        if animate_flag:
            axes[next_axes].plot(x1_LF[t_index], y1_LF[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(x2_LF[t_index], y2_LF[t_index], 'ro', markersize = '16')
        axes[next_axes].set_aspect(1)
        next_axes += 1
    
    if x_dot_vs_y_dot_plot_LF:
        plot_y_vs_x(x1_dot_LF,
                    y1_dot_LF,
                    axis_labels=x_dot_vs_y_dot_labels, 
                    label=rf'$orbit_{1}$', 
                    color = 'blue', title=r'Velocity Leapfrog Method', 
                    ax=axes[next_axes]) 
        plot_y_vs_x(x2_dot_LF, y2_dot_LF, axis_labels=x_dot_vs_y_dot_labels, 
                    label=rf'$orbit_{2}$', color = 'red', title=r'Velocity Leapfrog Method', 
                    ax=axes[next_axes])
        if animate_flag:
            axes[next_axes].plot(x1_dot_LF[t_index], y1_dot_LF[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(x2_dot_LF[t_index], y2_dot_LF[t_index], 'ro', markersize = '16')
        next_axes += 1

    if energy_plot_LF: 
        plot_y_vs_x(t_pts, energy_LF, 
                    axis_labels=energy_labels, 
                    title='Energy Leapfrog Method', 
                    color = 'green', 
                    ax=axes[next_axes])
        if animate_flag:
            axes[next_axes].plot(t_pts[t_index], energy_LF[t_index], 'go', markersize = '16')
        next_axes += 1
    
    # Do not show empty axes 
    if plot_num == 5:
        axes[5].axis('off')
    
    fig.tight_layout()
    
    return fig, axes


## Create widgets and display

In [65]:
# Widgets for the plot choice (plus a label out front)
plot_choice_w = Label(value='Which plots: ',layout=Layout(width='100px'))
def plot_choice_widget(on=True, plot_description=None):
    """Makes a Checkbox to select whether to show a plot."""
    return Checkbox(value=on, description=plot_description,
                  disabled=False, indent=False, layout=Layout(width='150px'))
x_vs_y_plot_w = plot_choice_widget(True, r'Position')
x_dot_vs_y_dot_plot_w = plot_choice_widget(False, r'Velocity')
energy_plot_w = plot_choice_widget(True, 'Energy')

x_vs_y_plot_LF_w = plot_choice_widget(True, r'Position Leapfrog')
x_dot_vs_y_dot_plot_LF_w = plot_choice_widget(False, r'Velocity Leapfrog')
energy_plot_LF_w = plot_choice_widget(True, 'Energy Leapfrog')

# Widgets for the pendulum parameters 
def float_widget(value, min, max, step, description, format):
    """Makes a FloatSlider with the passed parameters and continuous_update
       set to False."""
    slider_border = Layout(border='solid 1.0px')
    return FloatSlider(value=value,min=min,max=max,step=step,disabled=False,
                       description=description,continuous_update=False,
                       orientation='horizontal',layout=slider_border,
                       readout=True,readout_format=format)

G_w = float_widget(value=G, min=0.0, max=100., step=0.05,
                       description=r'Gravity:', format='.2f')
m1_w = float_widget(value=m1,min=0.0,max=100.,step=0.1,
                       description=r'Mass 1:', format='.2f')
m2_w = float_widget(value=m2, min=0, max=100.*np.pi, step=0.1,
                         description=r'Mass 2:', format='.2f')

# Widgets for the initial conditions
x1_0_w = float_widget(value=x1_0, min=0., max=2.*np.pi, step=0.01,
                        description=r'$(x_{1})_{0}$:', format='.1f')
x1_dot_0_w = float_widget(value=x1_dot_0, min=-10., max=10., step=0.1,
                            description=r'$(\dot x_{1})_0$:', format='.1f')
y1_0_w = float_widget(value=y1_0, min=0., max=2.*np.pi, step=0.01,
                        description=r'$(y_{1})_{0}$:', format='.1f')
y1_dot_0_w = float_widget(value=y1_dot_0, min=-10., max=10., step=0.1,
                            description=r'$(\dot y_{1})_0$:', format='.1f')

x2_0_w = float_widget(value=x2_0, min=0., max=2.*np.pi, step=0.01,
                        description=r'$(x_{2})_{0}$:', format='.1f')
x2_dot_0_w = float_widget(value=x2_dot_0, min=-10., max=10., step=0.1,
                            description=r'$(\dot x_{2})_0$:', format='.1f')
y2_0_w = float_widget(value=y2_0, min=0., max=2.*np.pi, step=0.01,
                        description=r'$(y_{2})_{0}$:', format='.1f')
y2_dot_0_w = float_widget(value=y2_dot_0, min=-10., max=10., step=0.1,
                            description=r'$(\dot y_{2})_0$:', format='.1f')


# Widgets for the plotting parameters
t_start_w = float_widget(value=t_start, min=0., max=100., step=.1,
                         description='t start:', format='.2f') 
t_end_w = float_widget(value=t_end, min=0.1, max=100., step=.1,
                       description='t end:', format='.2f')
delta_t_w = float_widget(value=delta_t, min=0.001, max=0.2, step=0.001,
                         description='delta t:', format='.3f')
plot_start_w = float_widget(value=0., min=0., max=300., step=5.,
                            description='start plotting:', format='.1f')

# Widgets for the animating
animate_flag_w = Checkbox(value=False, description='Animate',
                  disabled=False, indent=False, layout=Layout(width='100px'))
t_index_w = Play(interval=.01, value=0., min=1, max=1000, step=1, 
                      disabled=True, continuous_update=True,
                      description='press play', 
                      orientation='horizontal')

############## Begin: Explicit callback functions #######################

# Make sure that t_end is at least t_start + 50
def update_t_end(*args):
    if t_end_w.value < t_start_w.value:
        t_end_w.value = t_start_w.value + 50     
t_end_w.observe(update_t_end, 'value')
t_start_w.observe(update_t_end, 'value')

# Make sure that plot_start is at least t_start and less than t_end
def update_plot_start(*args):
    if plot_start_w.value < t_start_w.value:
        plot_start_w.value = t_start_w.value
    if plot_start_w.value > t_end_w.value:
        plot_start_w.value = t_end_w.value
plot_start_w.observe(update_plot_start, 'value')
t_start_w.observe(update_plot_start, 'value')
t_end_w.observe(update_plot_start, 'value')

# Only turn on play widget when animate is selected
def turn_on_play_widget(*args):
    if animate_flag_w.value is True:
        t_index_w.disabled = False
        t_pts = np.arange(t_start_w.value, t_end_w.value+delta_t_w.value, 
                          delta_t_w.value)  
        t_index_w.min = (np.fabs(t_pts-plot_start_w.value)).argmin() 
        t_index_w.max = len(t_pts) - 1
    else:
        t_index_w.disabled = True
animate_flag_w.observe(turn_on_play_widget, 'value')


############## End: Explicit callback functions #######################

# Set up the interactive_output widget 
plot_out = widgets.interactive_output(orbit_plots,
                          dict(
                          x_vs_y_plot=x_vs_y_plot_w,
                          x_dot_vs_y_dot_plot=x_dot_vs_y_dot_plot_w,
                          energy_plot=energy_plot_w,
                          x_vs_y_plot_LF=x_vs_y_plot_LF_w,
                          x_dot_vs_y_dot_plot_LF=x_dot_vs_y_dot_plot_LF_w,
                          energy_plot_LF=energy_plot_LF_w,
                          G=G_w,
                          m1=m1_w,
                          m2=m2_w,
                          x1_0=x1_0_w,
                          x1_dot_0=x1_dot_0_w,
                          y1_0=y1_0_w,
                          y1_dot_0=y1_dot_0_w,
                          x2_0=x2_0_w,
                          x2_dot_0=x2_dot_0_w,
                          y2_0=y2_0_w,
                          y2_dot_0=y2_dot_0_w,
                          t_start=t_start_w,
                          t_end=t_end_w, 
                          delta_t=delta_t_w,    
                          plot_start=plot_start_w,
                          animate_flag=animate_flag_w,
                          t_index=t_index_w)
                       )

# Now do some manual layout, where we can put the plot anywhere using plot_out
hbox0 = HBox([plot_choice_w, x_vs_y_plot_w, x_dot_vs_y_dot_plot_w,
              energy_plot_w])
hbox1 = HBox([plot_choice_w, x_vs_y_plot_LF_w, x_dot_vs_y_dot_plot_LF_w,
              energy_plot_LF_w ]) #  choice of what plots to show
hbox2 = HBox([G_w, m1_w, m2_w])  # parameters
hbox3 = HBox([x1_0_w, x1_dot_0_w, y1_0_w, y1_dot_0_w]) # initial conditions 1
hbox4 = HBox([x2_0_w, x2_dot_0_w, y2_0_w, y2_dot_0_w]) # initial conditions 2 
hbox5 = HBox([t_start_w, t_end_w, delta_t_w, plot_start_w]) # time and plot ranges
hbox6 = HBox([animate_flag_w, t_index_w])  # animate

# Set up tabs to organize the controls
tab_height = '70px'  # Fixed minimum height for all tabs.
tab0 = VBox([hbox2, hbox3, hbox4], layout=Layout(min_height=tab_height))
tab1 = VBox([hbox0, hbox1, hbox5], layout=Layout(min_height=tab_height))
tab2 = VBox([hbox6], layout=Layout(min_height=tab_height))

tab = Tab(children=[tab0, tab1, tab2])
tab.set_title(0, 'Physics')
tab.set_title(1, 'Plotting')
tab.set_title(2, 'Animate')

# Display widgets!
vbox = VBox([tab, plot_out])
display(vbox)

### By adjusting the masses and initial conditions using the widget menu  we can show that the problem reduces to the orbits considered in class if one of the bodies is very heavy and you are in its rest frame.  Also,  by comparing the energy spectra generated using both methods, we can see that using the Leapfrog method conserves energy while using the SciPy ODE solver does not.